In [14]:
%reload_ext autoreload
%autoreload 2

In [15]:
import sys
sys.path.append('../../resources')
from single_pad_transmon_pocket import TransmonPocket_Single

In [16]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [17]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [18]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

In [21]:
design.overwrite_enabled = True

options = dict()

q1 = TransmonPocket_Single(design,'Q1',options = dict(pad_width = '500um'))

  arr = construct_1d_object_array_from_listlike(values)



In [ ]:
gui.rebuild()
gui.autoscale()

  arr = construct_1d_object_array_from_listlike(values)



ocuments\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.pnge  WARNING: Could not create pixmap from c:\users�rkelium

ocuments\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.pnge  WARNING: Could not create pixmap from c:\users�rkelium

ocuments\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.pnge  WARNING: Could not create pixmap from c:\users�rkelium

ocuments\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.pnge  WARNING: Could not create pixmap from c:\users�rkelium

ocuments\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.pnge  WARNING: Could not create pixmap from c:\users�rkelium

ocuments\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.pnge  WARNING: Could not create pixmap from c:\users�rkelium

ocuments\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.pnge  WARNING: Could not create pixmap from c:\users�rkelium

ocuments\qiskit-metal\qiskit_metal\_gui\styles\metal_da

In [23]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [24]:
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 15,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': True,
 'solution_order': 'High',
 'solver_type': 'Iterative'}

In [10]:
c1.sim.run(components=['Q1'])#, open_terminations=[('Q1', 'readout'), ('Q1', 'bus1'), ('Q1', 'bus2')])
c1.sim.capacitance_matrix #455um

INFO 10:34PM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:34PM [load_ansys_project]: 	Opened Ansys App
INFO 10:34PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 10:34PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Berkelium/Documents/Ansoft/
	Project:   Project4
INFO 10:34PM [connect_design]: No active design found (or error getting active design).
INFO 10:34PM [connect]: 	 Connected to project "Project4". No design detected
INFO 10:34PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 10:34PM [connect_setup]: 	No design setup detected.
WARNING 10:34PM [connect_setup]: 	Creating Q3D default setup.
INFO 10:34PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 10:34PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 10:34PM [analyze]: Analyzing setup Setup
INFO 10:35PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Lo

,ground_main_plane,pad_top_Q1
ground_main_plane,150.56855,-105.22016
pad_top_Q1,-105.22016,108.50586


In [26]:
c1.sim.run(components=['Q1'])#, open_terminations=[('Q1', 'readout'), ('Q1', 'bus1'), ('Q1', 'bus2')])
c1.sim.capacitance_matrix #500um

INFO 04:37PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 04:37PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:37PM [analyze]: Analyzing setup Setup
INFO 04:38PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmpu1n9co6y.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:38PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmp47dgj6ou.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:38PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmp7ujq486b.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 04:38PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmphyenrs8o.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSi

,ground_main_plane,pad_top_Q1
ground_main_plane,159.70849,-114.59787
pad_top_Q1,-114.59787,118.14222


In [12]:
c1.setup.junctions = Dict({'Lj': 12.31, 'Cj': 2})


c1.run_lom()
c1.lumped_oscillator_all

ValueError: Capacitance matrix is not the right size

In [13]:
design.options

AttributeError: 'DesignPlanar' object has no attribute 'options'